In [2]:
import sys

In [3]:
sys.path.append('/Users/diegodavila/miniconda3/lib/python3.7/site-packages')

In [4]:
import flywheel
import datetime
import pytz
from time import gmtime, strftime

import pandas as pd
import numpy as np

fw = flywheel.Client()
proj = fw.lookup('bbl/ReproBrainChart')
sessions = proj.sessions()
sessions = [fw.get(x.id) for x in sessions]

/Users/diegodavila/miniconda3/lib/python3.7/site-packages/flywheel/flywheel.py:5758: UserWarning: Client version 9.0.3 does not match server version 11.3.2. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [5]:
def get_latest_fmriprep(session):
    
    if session.analyses:
        
        timezone = pytz.timezone("UTC")
        init_date = datetime.datetime(2018, 1, 1)
        latest_date = timezone.localize(init_date)

        latest_run = None

        for i in session.analyses:
            gear_name = i.gear_info['name']
            state = i.job.state
            date = i.created
            if 'fmriprep' in gear_name and date > latest_date and state =='complete':
                latest_date = date
                latest_run = i
    
        return(latest_run)

    else:
        return None

In [6]:
ses = sessions[0]
latest_fmriprep = get_latest_fmriprep(ses)
print(latest_fmriprep.created)
print(latest_fmriprep.gear_info)
print(latest_fmriprep.label)

2020-04-07 03:17:19.703000+00:00
{'category': 'analysis',
 'id': '5e8235058a6d118fb94f54d3',
 'name': 'fmriprep-hpc',
 'version': '0.3.2_20.0.5'}
fmriprep-hpc 04/06/2020 23:16:59


In [7]:
latest_fmriprep

{'created': datetime.datetime(2020, 4, 7, 3, 17, 19, 703000, tzinfo=tzutc()),
 'description': None,
 'files': [{'classification': {},
            'created': datetime.datetime(2020, 4, 7, 11, 39, 52, 114000, tzinfo=tzutc()),
            'hash': 'v0-sha384-dca211f0206d3c4076443c11995ae98faacb901ec56513e19596ea337db642719187080e4c58d521d9c3832924dcadb9',
            'id': '476ef38a-9693-4ea6-89c5-f1a89c6f2946',
            'info': {},
            'info_exists': None,
            'mimetype': 'application/zip',
            'modality': None,
            'modified': datetime.datetime(2020, 4, 7, 11, 39, 52, 114000, tzinfo=tzutc()),
            'name': 'sub-2231031108_5e8bf0bf67b1800dc5831d5c.html.zip',
            'origin': {'id': '5e8bf0bf67b1800dc5831d5d',
                       'method': None,
                       'name': None,
                       'type': 'job',
                       'via': None},
            'replaced': None,
            'size': 9102980,
            'tags': [],
    

In [8]:
latest_fmriprep.files

[{'classification': {},
  'created': datetime.datetime(2020, 4, 7, 11, 39, 52, 114000, tzinfo=tzutc()),
  'hash': 'v0-sha384-dca211f0206d3c4076443c11995ae98faacb901ec56513e19596ea337db642719187080e4c58d521d9c3832924dcadb9',
  'id': '476ef38a-9693-4ea6-89c5-f1a89c6f2946',
  'info': {},
  'info_exists': None,
  'mimetype': 'application/zip',
  'modality': None,
  'modified': datetime.datetime(2020, 4, 7, 11, 39, 52, 114000, tzinfo=tzutc()),
  'name': 'sub-2231031108_5e8bf0bf67b1800dc5831d5c.html.zip',
  'origin': {'id': '5e8bf0bf67b1800dc5831d5d',
             'method': None,
             'name': None,
             'type': 'job',
             'via': None},
  'replaced': None,
  'size': 9102980,
  'tags': [],
  'type': 'archive',
  'zip_member_count': None}, {'classification': {},
  'created': datetime.datetime(2020, 4, 7, 11, 39, 52, 114000, tzinfo=tzutc()),
  'hash': 'v0-sha384-8ba783dff3fb3c27e49554c2f17683af7f5a8c9be9f428ca6a673a63264917b811840344af4ce3396a0b57586205b1ee',
  'id': '03

In [9]:
fmriprep_out = [x for x in latest_fmriprep.files if 'html' not in x.name][0]
fmriprep_out

{'classification': {},
 'created': datetime.datetime(2020, 4, 7, 11, 39, 52, 114000, tzinfo=tzutc()),
 'hash': 'v0-sha384-8ba783dff3fb3c27e49554c2f17683af7f5a8c9be9f428ca6a673a63264917b811840344af4ce3396a0b57586205b1ee',
 'id': '03020359-6f56-4d20-9c42-4fde3b48cb41',
 'info': {},
 'info_exists': None,
 'mimetype': 'application/zip',
 'modality': None,
 'modified': datetime.datetime(2020, 4, 7, 11, 39, 52, 114000, tzinfo=tzutc()),
 'name': 'fmriprep_sub-2231031108_5e8bf0bf67b1800dc5831d5c.zip',
 'origin': {'id': '5e8bf0bf67b1800dc5831d5d',
            'method': None,
            'name': None,
            'type': 'job',
            'via': None},
 'replaced': None,
 'size': 1167264518,
 'tags': [],
 'type': 'archive',
 'zip_member_count': None}

In [10]:
xcp = fw.lookup('gears/xcpengine-fw')

In [11]:
print(xcp.gear['label'])
print(xcp.created)
print(xcp.gear['version'])

XCPENGINE: pipeline for processing of  structural and functional data.
2020-03-21 22:57:47.163000+00:00
0.0.1_1.00


In [12]:
print(xcp.gear['inputs'])
print()
print(xcp.gear['config'])

{'antsct': {'base': 'file', 'optional': True, 'description': 'Structural output directory'}, 'img': {'base': 'file', 'optional': True, 'description': 'asl or structural image '}, 'fmriprepdir': {'base': 'file', 'optional': True, 'description': 'FMRIprep output directory'}, 'custom_regressors': {'base': 'file', 'optional': True, 'description': 'custom regressors already convolved with hrf'}, 'm0': {'base': 'file', 'optional': True, 'description': 'M0 images for cbf calibration'}, 'taskfile': {'base': 'file', 'optional': True, 'description': 'eventtimes files with json  '}, 'designfile': {'base': 'file', 'description': 'designfile: list all modules to run'}, 'api_key': {'base': 'api-key'}}

{'analysis_type': {'default': 'xcp', 'type': 'string', 'description': ' analysis type: [fc,struc,task,cbf]'}, 'task_name': {'default': '', 'type': 'string', 'description': ' name of the task to perform either task regression or task '}}


In [13]:
designFile = [x for x in proj.files if x.name == "fc-36p_despike.dsn"][0]
designFile = proj.get_file(designFile.name)
print(designFile)

{'classification': {},
 'created': datetime.datetime(2020, 4, 20, 17, 12, 16, 674000, tzinfo=tzutc()),
 'hash': 'v0-sha384-33498e0d7d0fceda95cf1330633f0ab1aa1abedb5efaa700dbc597170d151696f043501e7bf7a8ce2813f866aa05d604',
 'id': '78ed41ad-f90e-4607-b9d6-b9541be1ffbe',
 'info': {},
 'info_exists': None,
 'mimetype': 'application/octet-stream',
 'modality': None,
 'modified': datetime.datetime(2020, 4, 20, 17, 12, 16, 286000, tzinfo=tzutc()),
 'name': 'fc-36p_despike.dsn',
 'origin': {'id': 'diegodav@upenn.edu',
            'method': None,
            'name': None,
            'type': 'user',
            'via': None},
 'replaced': None,
 'size': 4631,
 'tags': [],
 'type': None,
 'zip_member_count': None}


In [14]:
eventFile = [x for x in proj.files if x.name == "event1.txt"][0]
eventFile = proj.get_file(eventFile.name)
print(eventFile)

{'classification': {},
 'created': datetime.datetime(2020, 4, 20, 17, 13, 26, 393000, tzinfo=tzutc()),
 'hash': 'v0-sha384-e40ee79795268aa4db2f5a06e9d375728d5610c43cdc489bd7e3afae27dda4ee05a52a1ddef06b2a92915f72941eaf46',
 'id': '14710333-a211-4ef1-96dc-90f7cfd3a2d1',
 'info': {},
 'info_exists': None,
 'mimetype': 'text/plain',
 'modality': None,
 'modified': datetime.datetime(2020, 4, 20, 17, 13, 25, 969000, tzinfo=tzutc()),
 'name': 'event1.txt',
 'origin': {'id': 'diegodav@upenn.edu',
            'method': None,
            'name': None,
            'type': 'user',
            'via': None},
 'replaced': None,
 'size': 2102,
 'tags': [],
 'type': 'text',
 'zip_member_count': None}


In [15]:
id_list = pd.read_csv("/Users/diegodavila/Documents/BetweenWithinNetworks/Longitudinal/final_hashed.csv")
id_list

,bblid,reid
0,80557,628144066
1,81760,4108002155
2,82051,3896564969
3,82063,1645871327
4,82492,177928631
5,82790,579459470
6,83454,1963601913
7,83835,3984163767
8,83999,1088695522
9,84354,520579518


In [23]:
for ses in sessions:
    for name in id_list["reid"]:
        if (str(name) not in ses.subject.label) and (ses.label == "ses-PNC1"):
            print(ses.subject.label)
            break

sub-2231031108
sub-636218109
sub-2647862856
sub-2323472643
sub-1012530688
sub-619623685
sub-1654523276
sub-585579170
sub-1939119029
sub-736568905
sub-3239156026
sub-3497670658
sub-3957619959
sub-300988508
sub-2723385487
sub-3207921706
sub-470135601
sub-2355359789
sub-4231164226
sub-2470516380
sub-3131141015
sub-1738235220
sub-3671045126
sub-3190586154
sub-3810009389
sub-3177113419
sub-27876862
sub-316324995
sub-2038321583
sub-812909443
sub-3463353736
sub-1919055552
sub-361728233
sub-3726299659
sub-4099417558
sub-3950621480
sub-1871767977
sub-3175244541
sub-4033561173
sub-2249335715
sub-3370689672
sub-2404916865
sub-4197093898
sub-3317140614
sub-77390767
sub-375475352
sub-1057423710
sub-379170502
sub-1254730664
sub-2967570405
sub-646773252
sub-2110496117
sub-3373021317
sub-308228517
sub-3917203499
sub-3091738725
sub-2543780112
sub-1901910719
sub-250211156
sub-3340851327
sub-3917125611
sub-1794800030
sub-2733440164
sub-2097242394
sub-940718928
sub-3247943782
sub-1336976954
sub-876444031


sub-3467479455
sub-1996891106
sub-335003211
sub-2664902924
sub-881641391
sub-3719615756
sub-3866325191
sub-2994983988
sub-3901605230
sub-3902996821
sub-37329051
sub-3073794839
sub-2967555571
sub-1512440794
sub-628391776
sub-2192302358
sub-2170759229
sub-1781775187
sub-3731824395
sub-962036104
sub-2718695342
sub-941151156
sub-1496827612
sub-2117857991
sub-2184988827
sub-2499963942
sub-2285120424
sub-1403873511
sub-3381441522
sub-1453109530
sub-410808047
sub-3655426254
sub-2625914260
sub-415148142
sub-2256834144
sub-1566869901
sub-2066354189
sub-2715820110
sub-361460013
sub-1873729948
sub-4172833860
sub-662466380
sub-3914722336
sub-716537879
sub-3441116881
sub-986035435
sub-1384982728
sub-754064811
sub-3891584278
sub-130118047
sub-2804514247
sub-674306142
sub-1207361199
sub-3076915122
sub-488302404
sub-11407866
sub-1660839245
sub-4238772277
sub-2428446542
sub-952845911
sub-277985884
sub-3732485344
sub-1000393599
sub-4107507507
sub-864032752
sub-3150163478
sub-3182049011
sub-3761349403
su

In [24]:
xcp_runs = {}

for ses in sessions:
    for name in id_list["reid"]:
        if (str(name) not in ses.subject.label) and (ses.label == "ses-PNC1"):
            latest_fmriprep = get_latest_fmriprep(ses)
            if latest_fmriprep is not None:
                fmriprep_out = [x for x in latest_fmriprep.files if 'html' not in x.name][0]
                #fmriprep_out
                myinputs = {
                    #'eventfile': eventFile,
                    'designfile': designFile,
                    'fmriprepdir': fmriprep_out 
                }

                myconfig = {
                    'analysis type': 'func'
                }

                xcp_runs[ses.label] = xcp.run(analysis_label="XCP_func_36despike_{}".format(strftime("%Y-%m-%d %H:%M:%S", gmtime())), destination=ses, inputs=myinputs, config=myconfig)
                break



In [25]:
xcp_runs

{'ses-PNC1': '5ea310299e3def0709f11228'}